## LSTM 셀 구현
지금까지 RNN을 살펴보았다면 이제 LSTM 셀과 LSTM 계층을 살펴보겠습니다. 이번 예제에서 사용할 데이터셋은 MNIST입니다.

MNIST는 손으로 쓴 숫자 이미지(0~9까지 값을 갖는 고정 크기 이미지 (28px x 28px))들로 구성되어 있습니다.

먼저 필요한 라이브러리들을 호출합니다.

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset

from torch.autograd import Variable
from torch.nn import Parameter # 파라미터 목록을 가지고 있는 라이브러리
from torch import Tensor
from torch.utils.data import DataLoader

import torch.nn.functional as F

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(125)

print(device)


cuda


데이터에 대한 전처리를 합니다. 평균과 표준편차에 맞게 데이터를 정규화하기 위한 코드입니다.

In [3]:
mnist_fransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5),(1.0)) # 평균을 0.5, 표준편차를 1.0으로 데이터 정규화
])

`torchvision.datasets` 에서 제공하는 데이터셋 중 MNIST 데이터셋을 내려받습니다.

In [4]:
train_dataset = dataset.MNIST(root='./MNIST_DATASET', transform=mnist_fransform, train=True, download=True)
validation_dataset = dataset.MNIST(
    root='./MNIST_DATASET', transform=mnist_fransform, train=False, download=True)

test_dataset = dataset.MNIST(
    root='./MNIST_DATASET', transform=mnist_fransform, train=False, download=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 19395944.46it/s]


Extracting ./MNIST_DATASET\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 9663796.87it/s]

Extracting ./MNIST_DATASET\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 6983886.52it/s]


Extracting ./MNIST_DATASET\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./MNIST_DATASET\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_DATASET\MNIST\raw



* MNIST 데이터셋을 내려받기 위해 필요한 파라미터는 다음과 같습니다.

```py
train_dataset = dataset.MNIST(root='./MNIST_DATASET', transform=mnist_fransform, train=True, download=True)
```

* `root` : MNIST를 내려받을 위치 지정
* `transform` : 앞에서 정의했던 데이터 전처리 적용
* `train` : `True`로 설정할 경우 훈련용 데이터셋을 가져오지만, `False`로 설정할 경우 테스트용 데이터셋을 가져옵니다.
* `download` : `True`로 설정될 경우 내려받으려는 위치에 MNIST 파일이 없으면 내려받지만 파일이 있다면 내려받지 않습니다.

`dataloader`를 이용하여 내려받은 MNIST 파일을 메모리로 불러옵니다. 단, train_loader, valid_loader, test_loader 가 호출될 때 메모리로 불러온다는 점에 주의하세요

In [5]:
batch_size = 64

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
valid_loader = DataLoader(
    dataset=validation_dataset,
    batch_size=batch_size,
    shuffle=True
)
train_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)


배치 크기 및 에포크 등 변수에 대한 값을 지정합니다.

In [6]:
batch_size = 100
n_iters = 6000
num_epoch = n_iters / (len(train_dataset) / batch_size)
num_epoch = int(num_epoch)

print(num_epoch)
print(len(train_dataset))

10
60000


LSTM 셀에 대한 네트워크를 구축합니다. 모델의 전반적인 네트워크가 아닌 LSTM 셀에 집중한 네트워크입니다.

In [7]:
class LSTMcell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMcell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self): # 모델의 파라미터 초기화
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(1))

        gates = self.x2h(x) + self.h2h(hx)
        gates = gates.squeeze()
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = F.sigmoid(ingate) # 입력게이트에 시그모이드 활성화 함수 적용
        forgetgate = F.sigmoid(forgetgate) # 망각 게이트에 시그모이드 활성화 함수 적용
        cellgate = F.tanh(cellgate) # 셀 게이트에 하이퍼볼릭 탄젠트 활성화 함수 적용
        outgate = F.sigmoid(outgate) # 출력 게이트에 시그모이드 활성화 함수 적용

        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        return(hy, cy)

```py
self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
```
```py
self.h2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
```

에서 `4 * hidden_size`가 사용되고 있는 이유에 대해 생각해 볼 필요가 있습니다. 왜 은닉층의 뉴런/유닛에 4를 곱할까요?

그 답을 알기 위해서는 다음 그림을 먼저 이해해야 합니다.

![](../Static/403_1.jpg)

